In [ ]:
# useful additional packages
from qiskit_optimization.converters import QuadraticProgramToQubo
from itertools import permutations
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx

from qiskit_ibm_runtime import QiskitRuntimeService, Sampler as QiskitSampler
from qiskit_aer import Aer
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import TwoLocal
from qiskit_optimization.applications import Maxcut, Tsp
from qiskit.algorithms.minimum_eigensolvers import SamplingVQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA
from qiskit.utils import algorithm_globals
from qiskit_optimization.algorithms import MinimumEigenOptimizer

import matplotlib.pyplot as plt
import networkx as nx


def draw_graph(G, colors, pos):
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(G, node_color=colors, node_size=600,
                     alpha=0.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)
    # Write this figure to a file
    plt.savefig("graph.png")


def draw_tsp_solution(G, order, colors, pos):
    G2 = nx.DiGraph()
    G2.add_nodes_from(G)
    n = len(order)
    for i in range(n):
        j = (i + 1) % n
        G2.add_edge(order[i], order[j], weight=G[order[i]][order[j]]["weight"])
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(
        G2, node_color=colors, edge_color="b", node_size=600, alpha=0.8, ax=default_axes, pos=pos
    )
    edge_labels = nx.get_edge_attributes(G2, "weight")
    nx.draw_networkx_edge_labels(
        G2, pos, font_color="b", edge_labels=edge_labels)

    # Write this figure to a file
    plt.savefig("solution.png")


service = QiskitRuntimeService(channel="ibm_quantum", token="412d6f2dcdf46801b6e4f5b813decd7fd033a36effe42b44565c890f40b88032fdf15586aab594fafd92fa862136f78dfa9887f965d1058fad6eb6609f245017")

# Run on a simulator
backend = service.get_backend("ibmq_qasm_simulator")

# Create a Sampler object
sampler = QiskitSampler(backend)

# TRAVELING SALESMAN
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())

# Generating a graph of 3 nodes
n = 3
num_qubits = n**2
tsp = Tsp.create_random_instance(n)
adj_matrix = nx.to_numpy_array(tsp.graph)
print("distance\n", adj_matrix)

colors = ["r" for node in tsp.graph.nodes]
pos = [tsp.graph.nodes[node]["pos"] for node in tsp.graph.nodes]
draw_graph(tsp.graph, colors, pos)


# Brute force
def brute_force_tsp(w, N):
    a = list(permutations(range(1, N)))
    last_best_distance = 1e10
    for i in a:
        distance = 0
        pre_j = 0
        for j in i:
            distance = distance + w[j, pre_j]
            pre_j = j
        distance = distance + w[pre_j, 0]
        order = (0,) + i
        if distance < last_best_distance:
            best_order = order
            last_best_distance = distance
            print("order = " + str(order) + " Distance = " + str(distance))
    return last_best_distance, best_order


best_distance, best_order = brute_force_tsp(adj_matrix, n)
print(
    "Best order from brute force = "
    + str(best_order)
    + " with total distance = "
    + str(best_distance)
)


draw_tsp_solution(tsp.graph, best_order, colors, pos)


# Something something?


qp = tsp.to_quadratic_program()
print(qp.prettyprint())


qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(qp)
qubitOp, offset = qubo.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(qubitOp))

result = exact.solve(qubo)
print(result.prettyprint())

algorithm_globals.random_seed = 123
seed = 10598

optimizer = SPSA(maxiter=300)
ry = TwoLocal(qubitOp.num_qubits, "ry", "cz", reps=5, entanglement="linear")


vqe = SamplingVQE(ansatz=ry, optimizer=optimizer, sampler=sampler)


# # Submit the circuit to the sampler
# job = sampler.run(vqe)

# # Once the job is complete, get the result
# res = job.result()


# plot_histogram(
#     job.result().quasi_dists,
#     filename="histogram.png"
# )

result = vqe.compute_minimum_eigenvalue(qubitOp)

print("energy:", result.eigenvalue.real)
print("time:", result.optimizer_time)
x = tsp.sample_most_likely(result.eigenstate)
print("feasible:", qubo.is_feasible(x))
z = tsp.interpret(x)
print("solution:", z)
print("solution objective:", tsp.tsp_value(z, adj_matrix))
draw_tsp_solution(tsp.graph, z, colors, pos)

algorithm_globals.random_seed = 123
seed = 10598

# create minimum eigen optimizer based on SamplingVQE
vqe_optimizer = MinimumEigenOptimizer(vqe)

# solve quadratic program
result = vqe_optimizer.solve(qp)
print(result.prettyprint())

z = tsp.interpret(x)
print("solution:", z)
print("solution objective:", tsp.tsp_value(z, adj_matrix))
draw_tsp_solution(tsp.graph, z, colors, pos)


distance
 [[ 0. 69. 66.]
 [69.  0. 78.]
 [66. 78.  0.]]
order = (0, 1, 2) Distance = 213.0
Best order from brute force = (0, 1, 2) with total distance = 213.0
Problem name: TSP

Minimize
  69*x_0_0*x_1_1 + 69*x_0_0*x_1_2 + 66*x_0_0*x_2_1 + 66*x_0_0*x_2_2
  + 69*x_0_1*x_1_0 + 69*x_0_1*x_1_2 + 66*x_0_1*x_2_0 + 66*x_0_1*x_2_2
  + 69*x_0_2*x_1_0 + 69*x_0_2*x_1_1 + 66*x_0_2*x_2_0 + 66*x_0_2*x_2_1
  + 78*x_1_0*x_2_1 + 78*x_1_0*x_2_2 + 78*x_1_1*x_2_0 + 78*x_1_1*x_2_2
  + 78*x_1_2*x_2_0 + 78*x_1_2*x_2_1

Subject to
  Linear constraints (6)
    x_0_0 + x_0_1 + x_0_2 == 1  'c0'
    x_1_0 + x_1_1 + x_1_2 == 1  'c1'
    x_2_0 + x_2_1 + x_2_2 == 1  'c2'
    x_0_0 + x_1_0 + x_2_0 == 1  'c3'
    x_0_1 + x_1_1 + x_2_1 == 1  'c4'
    x_0_2 + x_1_2 + x_2_2 == 1  'c5'

  Binary variables (9)
    x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2

Offset: 7993.5
Ising Hamiltonian:
-1346.5 * IIIIIIIIZ
- 1346.5 * IIIIIIIZI
- 1346.5 * IIIIIIZII
- 1352.5 * IIIIIZIII
- 1352.5 * IIIIZIIII
- 1352.5 * IIIZIIII